In [ ]:
## 准备

In [ ]:
!pip install selenium -i https://pypi.sjtug.sjtu.edu.cn/simple/
#使用上海交通大学镜像源，防止selenium下载过慢导致失败
#selenium-最主要的爬虫包，模仿用户操作
import pandas as pd
#数据框处理
import time
#time.sleep()，强制等待
from selenium.webdriver.common.by import By
#选择器
from selenium.webdriver.support.ui import WebDriverWait
#等待器
from selenium.webdriver.support import expected_conditions as EC
#显式等待附件[1]
from selenium.webdriver.common.action_chains import ActionChains
#模拟悬停操作

In [ ]:
### 注释 
[1]  
Selenium WebDriver提供了两种等待机制：显式等待（Explicit Wait）和隐式等待（Implicit Wait）。它们都用于处理网页上的元素加载问题，但使用场景和工作原理有所不同。

In [ ]:
#### 显式等待
显式等待是一种更灵活、更强大、更推荐的方法。它允许你指定一个条件，WebDriver会等待直到该条件成立，或者达到一个设定的超时时间。显式等待通常使用`WebDriverWait`类和`expected_conditions`模块。

**特点**：
- 可以等待特定的条件，如元素出现、元素可见、元素可点击等。
- 可以设置超时时间，避免无限期等待。
- 可以提供更清晰的错误信息，因为WebDriver会在等待超时后抛出异常。

**示例**：
```python
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = ...  # 你的WebDriver实例
wait = WebDriverWait(driver, 10)  # 设置最大等待时间10秒
element = wait.until(EC.presence_of_element_located((By.ID, "myElement")))
```

In [ ]:
#### 隐式等待
隐式等待是Selenium早期版本中的一种等待机制，它在每次查找元素时自动等待，直到找到元素或达到默认的等待时间（通常是30秒）。

**特点**：
- 在查找元素时自动等待，不需要显式调用等待条件。
- 对于所有查找操作都是全局生效的。
- 等待时间是固定的，不能针对不同条件设置不同的等待时间。

**示例**：
```python
driver = ...  # 你的WebDriver实例
driver.implicitly_wait(10)  # 设置隐式等待时间为10秒
element = driver.find_element_by_id("myElement")
```

In [ ]:
#### 区别
1. **灵活性**：显式等待更灵活，可以为不同的条件设置不同的等待策略。
2. **效率**：显式等待通常更高效，因为它只在必要时才等待，而隐式等待会在每次元素查找时都进行等待。
3. **错误处理**：显式等待在超时时会抛出异常，便于调试；隐式等待则可能导致测试在超时后继续执行，增加了调试难度。
4. **推荐性**：显式等待是Selenium官方推荐的做法，因为它提供了更好的控制和更清晰的错误信息。

总的来说，显式等待提供了更多的控制和灵活性，使得测试脚本更加健壮和可靠。隐式等待虽然简单，但在大多数情况下，显式等待是更好的选择。

In [ ]:
## 主程序

目的是在一级页面读取多个可点击的元素并进入二级页面，在二级页面循环执行悬停操作并读取数据列表，最后统一存入一个文件夹并合成为一个文件导出。

In [ ]:
from selenium import webdriver
driver=webdriver.Chrome()
driver.get('http://baidu.com')
#打开对象浏览器
#可能要多等待一会

In [ ]:
all_handles=driver.window_handles
print(all_handles)
driver.switch_to.window(all_handles[-1])
current_handle=driver.current_window_handle
print(current_handle)
#打开多个窗口时句柄不会自动更新
#句柄获取及更新

In [ ]:
index=0
# 创建一个空的DataFrame，预定义两列的列名
df = pd.DataFrame(columns=["指标编码", "数据来源"])

while True:
    index+=1

    try:
        wait = WebDriverWait(driver, 10)
        # 使用locate elements方法查找所有具有class '自定义' 的元素
        elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'defaultColor')))
        print('已更新元素列表')
        # 在需要多级页面加载并返回的操作中，原先所获取的元素列表会报错，显示stale elements error
        # 所以当我们每一次回到上一级页面时都需要更新元素列表
        # 这也是我为什么不用for语句而用while语句
        wait.until(EC.element_to_be_clickable(elements[index]))
        elements[index].click()
        time.sleep(5)
        # 强制等待
        # 显示等待和隐式等待都是等到页面相对应的元素出现，或者可定位，或者可点击，就会执行下一步。
        # 但是往往页面中有多个重复的类元素，无法完全加载，就会执行下一步操作导致许多数据漏掉。
        # 所以一般操作的时候我选择强制等待，根据经验设置5~10秒钟。
    
    except:
        try:
            print("元素被其他元素遮挡，尝试使用JavaScript点击")
            # 使用JavaScript点击元素
            wait = WebDriverWait(driver, 10)
            elements = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'defaultColor')))
            driver.execute_script("arguments[0].click();", elements[index])
            # 点击第三或第4个元素时可能会显示该元素被遮挡无法点击，暂时还没有找到报错的原因
            # 这个问题可以用上面的js方法强制点击
        except:
             # 等待页面元素加载完成
            print('Java点击失败')
# 这里的try和except不是乱写的。如果在页面里面找不到可以区分的标签
# 有一些杂质元素也被包含在了目标对象里，就需要这种格式来避免`wait.until`类代码报错
    

# 由于页面中的一些元素，需要在悬停时显示悬浮窗(tooltip)后方可加载
# 所以接下来要执行模拟用户鼠标悬停操作
    time.sleep(10)
    wait = WebDriverWait(driver, 10)
    titles = wait.until(EC.visibility_of_all_elements_located((By.CLASS_NAME, "limit")))
    
    # 创建ActionChains对象
    actions = ActionChains(driver)

    # 遍历所有class为limit的元素，并模拟鼠标悬停
    count=0
    for title in titles:
        count+=1
        actions.move_to_element(title).perform()
        time.sleep(1)
        # 这里一秒左右的强制等待也应该考虑，可以提高数据获取的稳定性
    print(f"已扫过{count}条数据")

    # 定位所有class为ant-popover-inner-content的元素
    # 这假设了每个悬浮窗口的内容都被包裹在具有这个类的div中
    wait = WebDriverWait(driver, 10)
    time.sleep(5)
    popover_contents = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "ant-popover-inner-content")))
    
    line=0
    for popover_content in popover_contents:
        # 在每个.ant-popover-inner-content中，定位到.expressionshow元素
        expressionshow = popover_content.find_element(By.CLASS_NAME, "expressionshow")
        # 这里我曾经试过直接定位expression受这个元素，但是效果不太好
        # 先加载它的上一级菜单Pop over可以提高元素获取的完整度
        
        # 获取第一个p标签内的span标签文本，即编码
        first_p_span = expressionshow.find_element(By.XPATH, './/p[1]/span')
        first_item_code = first_p_span.get_attribute("textContent").strip()
    
        # 获取最后一个p标签内的文本，即填报内容
        last_p = expressionshow.find_element(By.XPATH, './/p[last()]')
        last_item_text = last_p.get_attribute("textContent").strip()
        # 假设 '数据来源：' 后面紧跟的是所需文本
        # 使用split方法分割文本，并取分割后的第二部分（索引为1）
        data_source_text = last_item_text.split("数据来源：")[1].strip()
        # 追加数据到DataFrame
        df = df._append({'指标编码': first_item_code, '数据来源': data_source_text}, ignore_index=True)
        line+=1
    print(f"已录入{line}条数据")

    driver.back()
    time.sleep(10)

In [ ]:
## 文件导出

In [ ]:
# 数据获取完成后还存在数据框内，需要导出到Excel文件
df.to_excel('patch3.xlsx', index=False)
# 这里默认的保存路径就是当前工作的列表
# 当前工作路径设置请参考另一篇jupyter使用心得
df = df.drop(df.index)
# 用来清除数据框内所有数据，方便执行下一次的循环

In [ ]:
# 现在需要将多次循环获得的矩阵数据放入同一个Excel表。
pip install pandas openpyxl

import os
folder_path="D:\\temp\\jupyter projects\\单病种质控"
# 这里填入所有矩阵数据存放的目标文件夹
output_path="D:\\temp\\jupyter projects\\单病种pro.xlsx"
# 这里填入你要生成的Excel表名称
# 如果该路径下已经存在同名文件会自动覆盖
files=[f for f in os.listdir(folder_path) if f.endswith('.xlsx')]

merged_df=pd.DataFrame()

for file in files:
    file_path=os.path.join(folder_path,file)
    df=pd.read_excel(file_path,usecols=['指标编码','数据来源'])
    merged_df=pd.concat([merged_df,df],ignore_index=True)
with pd.ExcelWriter(output_path,engine='openpyxl') as writer:
    writer.book.create_sheet('Sheet1')
    merged_df.to_excel(writer,index=False)
    # 一般来讲，这里执行完with语句后会自动关闭
    # 但有可能遇到python依旧占用该表而无法进行下一步操作的情况
print('done！')

In [ ]:
sessionid=driver.session_id
# 这里的session是我想控制已经打开的窗口
# 必须从程序拉起才能控制窗口吗？
# 待解答